# Multilevel Monte Carlo Solver (Tensorflow v.1.5)

Import packages, reset the computational graph, start interactive Tensorflow session and set random seed

In [153]:
import tensorflow as tf
from timeit import default_timer as timer
import numpy as np
tf.reset_default_graph()
sess = tf.InteractiveSession()
tf.set_random_seed(1)

Define Parameter of the PDE (constant coefficient parabolic equation)

In [154]:
with tf.variable_scope('PDE-Parameter'):
    d=100 #dimension
    mu=tf.zeros((1,d),name='Mu') #(alternativ: tf.zeros((1,d),name='Mu') for explicit solution, 0.2*tf.ones((1,d),name='Mu'))
    sigma=tf.multiply(1/d**0.5,tf.ones((d,d)),name='Sigma') #Sigma TRANSPOSED #(alternativ: np.eye(d), 0.5*np.random.randint(-2,2,(d,d)), np.random.uniform(-2,2,(d,d)))

Define Parameter of the Algorithm

In [155]:
T=1.0 #time, where the solution is evaluated
a=0.0 
b=1.0 #approx. the solution of the PDE in [a,b]^d
l_max=0 #2^(l_max) is the number of time steps in the finest level
l_min=0 #2^(l_min) is the number of time steps in the coarsest level
#n_hidlayer_fine=1
#n_neurons_fine=[16]
n_hidlayer=5 #number of hidden layers for the neural nets
n_neurons=[d,256,128,64,16] #number of neurons for the hidden layers
K=16 #batch/sample multiplicator for the different levels
n_valid=10000 #number of samples for validation of the expected value/loss
n_acc=50 #number of samples for measuring the accuracy
valid_steps=500 #every valid_steps the loss is computed
#start_learn_rate_coarse=1e-3 #starting learning rate of the gradient descent
start_lr=1e-3
decay_steps=500 #learning rate decays exponentially 
w_initializer=tf.contrib.layers.variance_scaling_initializer() #initializer of the weights (alternativ: tf.contrib.layers.xavier_initializer(uniform=True) for Sigmoid, tf.constant_initializer(0), tf.contrib.layers.variance_scaling_initializer() for ReLu, tf.truncated_normal_initializer(0,1))
#weight_initializer_fine=tf.contrib.layers.variance_scaling_initializer()
b_initializer=tf.zeros_initializer() #initializer of the biases (alternativ: tf.zeros_initializer(), tf.constant_initializer(0.01) for ReLu)
#bias_initializer_fine=tf.zeros_initializer()
activation=tf.nn.sigmoid #activation function (alternativ: tf.nn.sigmoid, tf.nn.elu, tf.nn.relu, tf.nn.tanh)
adam_eps=1e-7 #epsilon of the Adam optimizer

Functions for defining a neuronal network, attaching summaries to the Tensors (for TensorBoard visualization), feeding with data, calculating the loss and optimizing

In [156]:
def variable_summaries(var): 
    with tf.variable_scope('Summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('Average', mean)
        tf.summary.histogram('Histogram', var)

def phi(x,lvl):
    with tf.variable_scope('Euler-Scheme'):
        with tf.variable_scope('Fine_Realization'):
            N_fine=tf.constant(2**lvl,name='Steps_fine',dtype=tf.int32) 
            h_fine=tf.divide(T,tf.cast(N_fine,tf.float32),name='Step-Size_fine')
            dw_fine=tf.random_normal(shape=[N_fine,tf.shape(x)[0],d],mean=0.0,stddev=tf.sqrt(h_fine),name='DW')
            count=tf.constant(0,name='Count')
            def scheme_fine(i,realisation):
                realisation+=mu*h_fine+tf.matmul(dw_fine[i,:,:],sigma)
                return [i+1,realisation]
            _, y_fine=tf.while_loop(lambda i,realisation: i<N_fine, scheme_fine, loop_vars=[count,x],name='Euler-Loop_fine')
        if lvl==l_min:
            phi=tf.reduce_sum(y_fine**2, axis=1, keepdims=True, name='Phi')
            return phi
        else:
            with tf.variable_scope('Coarse_Realization'):
                N_coarse=tf.constant(2**(lvl-1),name='Steps_coarse',dtype=tf.int32)
                h_coarse=tf.divide(T,tf.cast(N_coarse,tf.float32),name='Step-Size_coarse')
                dw_coarse=dw_fine[0::2,:,:]+dw_fine[1::2,:,:]
                def scheme_coarse(i,realisation):
                    realisation+=mu*h_coarse+tf.matmul(dw_coarse[i,:,:],sigma)
                    return [i+1,realisation]                 
                _, y_coarse=tf.while_loop(lambda i,realisation: i<N_coarse, scheme_coarse, loop_vars=[count,x],name='Euler-Loop_coarse')
            phi=tf.subtract(tf.reduce_sum(y_fine**2, axis=1, keepdims=True),tf.reduce_sum(y_coarse**2, axis=1, keepdims=True),name='Phi')
            return phi
            
def nn(input_layer, num_hidlayer, num_neurons, level, weight_initializer, bias_initializer, start_learn_rate, training):
    name_suffix=str(level)
    with tf.variable_scope('Network_'+name_suffix):           
        with tf.variable_scope('Target'):
            is_validation=tf.placeholder(tf.bool,name='Is_Validation') #boolean for batch normalization and input decision
            z=tf.cond(is_validation,lambda: tf.placeholder(tf.float32,[None,1],name='Z-Input'),lambda: phi(input_layer,level),name='Network-Target')
        with tf.variable_scope('Normalization'):
            prev_output=input_layer-(a+b)/2 #shift to zero mean for better performance
        for n in range(num_hidlayer):
            with tf.variable_scope('Hidden_Layer%d' %(n+1)):
                prev_output=tf.contrib.layers.fully_connected(prev_output,num_neurons[n],activation_fn=activation,normalizer_fn=tf.contrib.layers.batch_norm,normalizer_params={'is_training':training,'updates_collections':'updates'},weights_initializer=weight_initializer)
                #prev_output=tf.contrib.layers.fully_connected(prev_output,num_neurons[n],activation_fn=activation,weights_initializer=weight_initializer,biases_initializer=bias_initializer)
                variable_summaries(prev_output)
        with tf.variable_scope('Output_Layer'):
            output=tf.contrib.layers.fully_connected(prev_output,1,activation_fn=None,normalizer_fn=None,weights_initializer=weight_initializer,biases_initializer=bias_initializer)
            variable_summaries(output)
        with tf.variable_scope('Losses'):
            #delta=tf.clip_by_value(z-output,-100.0,100.0) #clipped difference between output and labels 
            #loss=tf.reduce_mean(delta**2, name='Loss') #mean squared error (alternativ: loss=tf.losses.mean_squared_error(z,output))
            loss=tf.reduce_mean((z-output)**2, name='Loss')
            tf.summary.scalar('Loss-Summary', loss)
    with tf.name_scope('Train_'+name_suffix):            
        global_step=tf.Variable(0, trainable=False, name='Global_Step') #counts the number of training steps
        learn_rate=tf.train.exponential_decay(start_learn_rate, global_step,decay_steps, 0.99, staircase=True, name='Learn_Rate')
        optimizer=tf.train.AdamOptimizer(learn_rate,epsilon=1e-7, name='Adam') #tf.train.RMSPropOptimizer(learn_rate, centered=True, name='RMSProp') #tf.train.AdagradOptimizer(learn_rate, name='Adagrad') #MomentumOptimizer(learn_rate,0.5,name='Momentum',use_nesterov=True) #(alternativ: lambda lr: tf.train.AdamOptimizer(lr,epsilon=1e-7, name='Adam'))
        var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Network_'+name_suffix)
        update_ops = tf.get_collection('updates', scope='Network_'+name_suffix)
        with tf.control_dependencies(update_ops):
            training=optimizer.minimize(loss,global_step=global_step,var_list=var_list,name='Minimizer')
        summaries=tf.get_collection(tf.GraphKeys.SUMMARIES, scope='Network_'+name_suffix)
        merged=tf.summary.merge(summaries, name='Merged') #run all summaries at once
    return output, z

def uniform_input():
    batch_size=tf.placeholder(tf.int32,shape=[],name='Batchsize')
    return tf.random_uniform([batch_size,d],minval=a,maxval=b,name='Xi-Input')

def build_model():
    with tf.variable_scope('Input'):
        is_training=tf.placeholder(tf.bool,name='Is_Training') #boolean for batch normalization and input decision
        nn_input=tf.cond(is_training,lambda: uniform_input(),lambda: tf.placeholder(tf.float32,[None,d],name='X-Input'),name='Network-Input')
    out=[nn(nn_input,n_hidlayer,n_neurons,l, w_initializer, b_initializer, start_lr, is_training) for l in range(l_min,l_max+1)]
    nn_outputs=[lst[0] for lst in out]
    phi_outputs=[lst[1] for lst in out]
    with tf.variable_scope('Accuracy'):
        u_MC=tf.add_n(phi_outputs,name='U_MC')
        u_MC_mean=tf.reduce_mean(u_MC,name='U_MC_Mean')
        u_real=tf.placeholder(tf.float32,[None,1],name='U_Real') #Input for the actual solution
        u_approx=tf.add_n(nn_outputs,name='U_Approximation') #Function for evaluation of the approximation (of the function u(T,x))
        abs_diff=tf.abs(u_approx-u_real,name='Absolute-Error')
        max_error=tf.reduce_max(abs_diff,name='Max-Error')
        tf.summary.scalar('Max_Error-Summary', max_error)
        l2_error=tf.sqrt(tf.reduce_mean(abs_diff**2)*(b-a)**d,name='L2-Error')
        tf.summary.scalar('L2_Error-Summary', l2_error)
        summaries_acc=tf.get_collection(tf.GraphKeys.SUMMARIES, scope='Accuracy')
        merged_acc=tf.summary.merge(summaries_acc, name='Merged_Acc') #run all summaries at once

def valid_accuracy_data(num_validation,num_accuracy):
    dictionary={'Input/Is_Training:0': True, 'Input/Network-Input/Batchsize:0': num_accuracy}
    accuracy_data=[sess.run('Input/Network-Input/Merge:0',feed_dict=dictionary)]
    dictionary['Input/Network-Input/Batchsize:0']=num_validation
    dictionary.update({'Network_'+str(l)+'/Target/Is_Validation:0': False for l in range(l_min,l_max+1)})
    fetch=['Input/Network-Input/Merge:0']
    fetch.append(['Network_'+str(l)+'/Target/Network-Target/Merge:0' for l in range(l_min,l_max+1)])
    validation_data=sess.run(fetch,feed_dict=dictionary)
    dictionary['Input/Is_Training:0']=False    
    del dictionary['Input/Network-Input/Batchsize:0']
    MC_mean=np.empty((num_accuracy,1))
    for sample in range(num_accuracy):
        dictionary.update({'Input/Network-Input/X-Input:0': np.tile(accuracy_data[0][sample],(5000,1))})
        MC_mean[sample]=sess.run('Accuracy/U_MC_Mean:0',feed_dict=dictionary)    
    accuracy_data.append(MC_mean)
    return validation_data, accuracy_data

#less input batchsize for higher levels!!!
    
def trainNN(level,n_iterations,validation_data,accuracy_data,batch_sizes):
    name_suffix=str(level)
    scope='Network_'+name_suffix+'/'
    scope_train='Train_'+name_suffix+'/'
    scope_target=scope+'Target/'
    scope_z_target=scope_target+'Network-Target/'
    valid_dictionary={'Input/Network-Input/X-Input:0': validation_data[0], scope_z_target+'Z-Input:0': validation_data[1][level], 'Input/Is_Training:0': False, scope_target+'Is_Validation:0': True}
    accuracy_dictionary={'Input/Network-Input/X-Input:0': accuracy_data[0], 'Input/Is_Training:0': False, 'Accuracy/U_Real:0': accuracy_data[1]}
    glob_iterations=sess.run(scope_train+'Global_Step:0')
    for iteration in range(glob_iterations,glob_iterations+n_iterations): #for all iterations
        if ((iteration)%valid_steps)==0: #output loss with valid_K batchsize
            summary, rate, valid_loss=sess.run([scope_train+'Merged/Merged:0',scope_train+'Learn_Rate:0', scope+'Losses/Loss:0'], feed_dict=valid_dictionary)
            summary_acc, l2_err, max_err=sess.run(['Accuracy/Merged_Acc/Merged_Acc:0','Accuracy/L2-Error:0', 'Accuracy/Max-Error:0'], feed_dict=accuracy_dictionary)
            writer[level-l_min].add_summary(summary, iteration) 
            writer[level-l_min].add_summary(summary_acc, iteration)  
            print('Network: %d, Iteration: %d, Loss: %.8f, Max. Error: %.4f, L2-Error: %.4f, Learning Rate: %.1E' %(level,iteration,valid_loss,max_err,l2_err,rate))
        sess.run(scope_train+'Minimizer:0', feed_dict={'Input/Is_Training:0': True, scope_target+'Is_Validation:0': False, 'Input/Network-Input/Batchsize:0': batch_sizes})

Build the neuronal networks, initialize variables and prepare summaries

In [157]:
build_model()
print('Model build!')
valid_data, acc_data=valid_accuracy_data(n_valid,n_acc)
print('Data for validation and accuracy measurements computed!')
tf.global_variables_initializer().run() #initializiation of the variables
print('Variables initialized!')
count=0 #find a new directory for the summary logs
while True:
    count+=1
    if not tf.gfile.Exists('logs/log'+str(count)):
        dir='logs/log'+str(count)
        break
writer = [tf.summary.FileWriter(dir+'/network_'+str(l)) for l in range(l_min,l_max+1)] #write summaries (takes some time to execute)
writer[0].add_graph(sess.graph) 
print('Tensorboard summaries prepared!')

Model build!
Data for validation and accuracy measurements computed!
Variables initialized!
Tensorboard summaries prepared!


Train the networks

In [158]:
n_iter=40000 #number of gradient descents
start = timer()
for l in range(l_min,l_max+1):    
    trainNN(l,n_iter,valid_data,acc_data,128*2**(l_max+l_min-l)) #K
end = timer()
print('Elapsed minutes to train the networks: ',(end - start)/60)  

Network: 0, Iteration: 0, Loss: 46553.92968750, Max. Error: 142.9282, L2-Error: 133.9649, Learning Rate: 1.0E-03
Network: 0, Iteration: 500, Loss: 44830.26562500, Max. Error: 136.2139, L2-Error: 127.2600, Learning Rate: 9.9E-04
Network: 0, Iteration: 1000, Loss: 42820.62109375, Max. Error: 127.9701, L2-Error: 118.9290, Learning Rate: 9.8E-04
Network: 0, Iteration: 1500, Loss: 40844.11328125, Max. Error: 119.0546, L2-Error: 110.1335, Learning Rate: 9.7E-04
Network: 0, Iteration: 2000, Loss: 39028.14062500, Max. Error: 110.2086, L2-Error: 101.3641, Learning Rate: 9.6E-04
Network: 0, Iteration: 2500, Loss: 37398.22656250, Max. Error: 101.5493, L2-Error: 92.7330, Learning Rate: 9.5E-04
Network: 0, Iteration: 3000, Loss: 36021.19140625, Max. Error: 93.5751, L2-Error: 84.7646, Learning Rate: 9.4E-04
Network: 0, Iteration: 3500, Loss: 34825.08203125, Max. Error: 85.9648, L2-Error: 77.1582, Learning Rate: 9.3E-04
Network: 0, Iteration: 4000, Loss: 33780.62109375, Max. Error: 78.6667, L2-Error:

KeyboardInterrupt: 

Optional: Train further selected networks

In [ ]:
l=0
n_iter=5000
batch_sz=1000 #K*2**(l_max+l_min-level)
trainNN(l,n_iter,valid_data,acc_data,batch_sz)

## Numerical experiments

Test accuracy against analytic solution (only for mu=0!!)  

In [159]:
def real_value(X):
    sigma_matrix=np.transpose(sess.run('PDE-Parameter/Sigma:0'))
    return (np.sum(X**2,axis=1,keepdims=True)+T*np.trace(sigma_matrix@np.transpose(sigma_matrix)))

def max_l2_analytic_error(n_tests): 
    dictionary={'Input/Is_Training:0': True, 'Input/Network-Input/Batchsize:0': n_tests}
    xi=sess.run('Input/Network-Input/Merge:0',feed_dict=dictionary)
    del dictionary['Input/Network-Input/Batchsize:0']
    real_u=real_value(xi)
    dictionary.update({'Input/Network-Input/X-Input:0': xi, 'Input/Is_Training:0': False, 'Accuracy/U_Real:0': real_u})
    max_error, l2_error=sess.run(['Accuracy/Max-Error:0','Accuracy/L2-Error:0'],feed_dict=dictionary)
    return max_error, l2_error

Test accuracy against (Multilevel) Montecarlo sampling (and actual solution if mu=0)

In [160]:
max_analytic, l2_analytic=max_l2_analytic_error(500000)
print('Max. Error: %.4f L2_Error: %.4f' %(max_analytic,l2_analytic))

Max. Error: 15.5524 L2_Error: 2.6893


In [161]:
def u_eval(X):
    return sess.run('Accuracy/U_Approximation:0', feed_dict={'Input/Network-Input/X-Input:0': X, 'Input/Is_Training:0': False})

In [162]:
x_0=np.zeros((1,d))
x_0[0,0]=0.2
tests=[np.ones((1,d)),x_0,np.random.uniform(a,b,(1,d)),np.zeros((1,d)),0.1*np.ones((1,d)),0.2*np.ones((1,d)),0.3*np.ones((1,d)),0.4*np.ones((1,d)),0.5*np.ones((1,d)),0.6*np.ones((1,d)),np.array([np.arange(1,d+1,1)**(-1.0)]),np.random.uniform(a,b,(1,d))]
for count, test in enumerate(tests, start=1):
    print('\n test: ',count,', x=',test,'\n')
    print('Real value (only for mu=0): ',real_value(test))
    #print('Monte-Carlo:                ',Montecarlosampling(5000,test))
    #print('Multilevel Monte-Carlo:     ',MLMontecarlosampling(128,test))
    print('Neuronal Network approx.:   ',u_eval(test))
    


 test:  1 , x= [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]] 

Real value (only for mu=0):  [[199.99993134]]
Neuronal Network approx.:    [[133.75131]]

 test:  2 , x= [[0.2 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]] 

Real value (only for mu=0):  [[100.03993134]]
Neuronal Network approx.:    [[126.71176]]

 test:  3 , x= [[0.28872775 0.84648324 0.12352756 0.10981383 0.1

Optional: Plot the solution (if d=1 or d=2)

In [ ]:
if d==1:
    import matplotlib.pyplot as plt
    X=np.arange(a, b, 0.01)
    Y=eval_u(np.transpose(np.array([X])))
    plt.plot(X,Y, 'r')
    plt.show()
if d==2:
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    fig=plt.figure()
    ax=fig.gca(projection='3d')
    n=20
    X,Y=np.meshgrid(np.arange(a, b, (b-a)/n), np.arange(a, b, (b-a)/n))
    Z=np.zeros(X.shape)
    Z_sol=np.zeros(X.shape)
    for i in range(n):
        for j in range(n):
            Z[i,j]=eval_u(np.array([[X[i,j],Y[i,j]]]))
            Z_sol[i,j]=real_value(np.array([[X[i,j],Y[i,j]]]))
    ax.plot_surface(X, Y, Z)
    ax.plot_surface(X, Y, Z_sol)
    plt.show()
    print('Neural Network Approx.: Blue, Exact Solution: Orange')

Close the tensorflow session

In [ ]:
sess.close()

Optional: Show the Neuronal networks and Summaries on tensorboard

In [ ]:
!tensorboard --logdir="logs" --port=6006

now open: http://localhost:6006/ or http://PC-NAME:6006/ (z.B.: http://Julius-PC:6006/) restart the kernel afterwards

In [91]:
#dictionary={'Input/Is_Training:0': True, 'Input/Network-Input/Batchsize:0': 1}
#accuracy_data=[sess.run('Input/Network-Input/Merge:0',feed_dict=dictionary)]
#dictionary['Input/Network-Input/Batchsize:0']=num_validation
dictionary=dict()
num_accuracy=1
accuracy_data=[[np.ones((1,d))]]
dictionary.update({'Network_'+str(l)+'/Target/Is_Validation:0': False for l in range(l_min,l_max+1)})
#fetch=['Input/Network-Input/Merge:0']
#fetch.append(['Network_'+str(l)+'/Target/Network-Target/Merge:0' for l in range(l_min,l_max+1)])
#validation_data=sess.run(fetch,feed_dict=dictionary)
dictionary['Input/Is_Training:0']=False    
#del dictionary['Input/Network-Input/Batchsize:0']
MC_mean=np.empty((num_accuracy,1))
for sample in range(num_accuracy):
    dictionary.update({'Input/Network-Input/X-Input:0': np.tile(accuracy_data[0][sample],(100000,1))})
    MC_mean[sample]=sess.run('Accuracy/U_MC_Mean:0',feed_dict=dictionary)    
accuracy_data.append(MC_mean)
accuracy_data

[[array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1.]])], array([[100.97367859]])]